In [ ]:
! nvidia-smi

Fri Jul 22 02:19:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    38W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Mount to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import necessary libraries
import os

import numpy as np
import pandas as pd

import sklearn
import sklearn.metrics
import sklearn.preprocessing

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers

In [ ]:
# Seed to reproduce results
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
# Change root directory
% cd /content/drive/MyDrive/NLP-Project/

/content/drive/MyDrive/NLP-Project


In [ ]:
# Load data
df_train = pd.read_csv(f"{os.getcwd()}/dataset/train_dataset.csv")
df_test = pd.read_csv(f"{os.getcwd()}/dataset/test_dataset.csv")

In [ ]:
df_train

,text,label_name,label_id
0,kinh_tế trung_quốc chìm sâu suy_thoái chống dị...,kinh-doanh-tai-chinh,4
1,tòa_án choi tiếp_cận cook phạm_vi m cấm liên_l...,cong-nghe,0
2,thủ_tục nhập_cảnh xứ chùa vàng đơn_giản thanh_...,doi-song,1
3,ô dề hành_vi lố_lăng khẩu_ngữ thông_dụng đời_s...,doi-song,1
4,hãng hàng dừng hoạt_động khắp thông_báo hôm tu...,the-gioi,6
...,...,...,...
5140,kết_thúc quý i tổng_doanh_thu alibaba kỳ năm_n...,kinh-doanh-tai-chinh,4
5141,zhang làng tháp_hà tỉnh hà_nam miền trung_trun...,doi-song,1
5142,truyền_thông trung_quốc xưởng đóng tàu giang_n...,the-gioi,6
5143,rạng hà_nội ban lãnh_đạo psg thông_báo chia_ta...,the-thao,7


In [ ]:
# Label One-Hot-Encoding
label_encoded = sklearn.preprocessing.OneHotEncoder(sparse=False).fit_transform(pd.DataFrame([*list(df_train.label_id), *list(df_test.label_id)]))

In [ ]:
# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((df_train.text, label_encoded[:len(df_train)]))
test_dataset = tf.data.Dataset.from_tensor_slices((df_test.text, label_encoded[len(df_train):]))

# Batches
BUFFER_SIZE = 10000
BATCH_SIZE = 128

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Encoder Vocab Size
VOCAB_SIZE = 10000

# Encoder
encoder = layers.TextVectorization(VOCAB_SIZE)

# Adapt TextVectorization for Train text to sets the layer's vocabulary
encoder.adapt(train_dataset.map(lambda text, label: text))

# Vocab
print(f"Vocab_size: {len(encoder.get_vocabulary())}")

Vocab_size: 10000


In [ ]:
# Model
model = Sequential([
    encoder,
    layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=128,
        mask_zero=True
        ),
    layers.Bidirectional(layers.GRU(64, return_sequences = True)),
    layers.Dropout(0.3),
    layers.Bidirectional(layers.GRU(32)),
    layers.Dropout(0.3),
    layers.Dense(32, activation = "relu"),
    layers.Dense(8, activation = "softmax")])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 128)         1280000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        74496     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               31104     
 nal)                                                            
                                                        

In [ ]:
# Compile
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(5 * 1e-3),
    metrics='acc'
)

# Train 
history_training = model.fit(
    train_dataset, 
    epochs=50,
    validation_data = test_dataset
)

Epoch 1/50
41/41 [==============================] - 49s 715ms/step - loss: 0.4042 - acc: 0.1984 - val_loss: 0.3184 - val_acc: 0.3341
Epoch 2/50
41/41 [==============================] - 26s 651ms/step - loss: 0.2261 - acc: 0.6328 - val_loss: 0.1631 - val_acc: 0.7669
Epoch 3/50
41/41 [==============================] - 27s 659ms/step - loss: 0.0933 - acc: 0.8871 - val_loss: 0.1406 - val_acc: 0.7964
Epoch 4/50
41/41 [==============================] - 27s 668ms/step - loss: 0.0409 - acc: 0.9549 - val_loss: 0.1508 - val_acc: 0.8127
Epoch 5/50
41/41 [==============================] - 23s 558ms/step - loss: 0.0173 - acc: 0.9823 - val_loss: 0.1680 - val_acc: 0.8159
Epoch 6/50
41/41 [==============================] - 24s 597ms/step - loss: 0.0070 - acc: 0.9942 - val_loss: 0.1918 - val_acc: 0.8174
Epoch 7/50
41/41 [==============================] - 23s 560ms/step - loss: 0.0035 - acc: 0.9975 - val_loss: 0.2041 - val_acc: 0.8112
Epoch 8/50
41/41 [==============================] - 22s 551ms/step - 

In [ ]:
# Evaluation
train_loss, train_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)

# Prediction
preds = model.predict(df_test.text)
preds = tf.math.argmax(preds, -1)

# Report
report = sklearn.metrics.classification_report(df_test.label_id, preds, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report

11/11 [==============================] - 2s 196ms/step - loss: 0.2620 - acc: 0.8415


,precision,recall,f1-score,support
0,0.877193,0.857143,0.867052,175.000000
1,0.647436,0.660131,0.653722,153.000000
2,0.853846,0.822222,0.837736,135.000000
3,0.841772,0.852564,0.847134,156.000000
4,0.843931,0.802198,0.822535,182.000000
5,0.914894,0.905263,0.910053,190.000000
6,0.848958,0.885870,0.867021,184.000000
7,0.899160,0.955357,0.926407,112.000000
accuracy,0.841492,0.841492,0.841492,0.841492
macro avg,0.840899,0.842593,0.841457,1287.000000
